# F1 Score Improvement Experiments

Test multiple strategies to improve F1 beyond baseline 62.54%:
1. Feature selection (reduce noisy TF-IDF features)
2. Add title TF-IDF features
3. Ensemble methods
4. Combined approaches

**Baseline:** LogisticRegression with class_weight='balanced' + threshold=0.54 → F1=62.54%

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## 1. Load Data

In [ ]:
X_train = pd.read_pickle('../data/features/X_train_temporal.pkl')
X_test = pd.read_pickle('../data/features/X_test_temporal.pkl')
y_train = pd.read_pickle('../data/features/y_train_cls_temporal.pkl')
y_test = pd.read_pickle('../data/features/y_test_cls_temporal.pkl')

# Load metadata for title extraction
metadata_train = pd.read_pickle('../data/features/metadata_train.pkl')
metadata_test = pd.read_pickle('../data/features/metadata_test.pkl')

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")
print(f"\nClass distribution:")
print(y_train.value_counts())

## 2. Baseline Model

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, threshold=0.54, name="Model"):
    """Train and evaluate a model with optimal threshold"""
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    results = {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }
    
    return results, model

print("=" * 60)
print("BASELINE - All 5,027 features")
print("=" * 60)

baseline_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

baseline_results, _ = evaluate_model(baseline_model, X_train, y_train, X_test, y_test, name="Baseline")

for metric, value in baseline_results.items():
    if metric != 'Model':
        print(f"{metric:12s}: {value*100:.2f}%")

results_list = [baseline_results]

## 3. Experiment 1: Feature Selection (Top 2000 Features)

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 1 - Feature Selection (5027 → 2000 features)")
print("=" * 60)

# Select top 2000 features using chi-squared test
selector = SelectKBest(chi2, k=2000)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

print(f"\nReduced from {X_train.shape[1]} to {X_train_selected.shape[1]} features")

exp1_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

exp1_results, _ = evaluate_model(exp1_model, X_train_selected, y_train, X_test_selected, y_test, name="Feature Selection (k=2000)")

print("\nResults:")
for metric, value in exp1_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp1_results)

## 4. Experiment 2: Add Title Features

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 2 - Add Title TF-IDF Features (500 features)")
print("=" * 60)

# Extract title features
title_vectorizer = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.8
)

title_train = title_vectorizer.fit_transform(metadata_train['Title'].fillna(''))
title_test = title_vectorizer.transform(metadata_test['Title'].fillna(''))

# Combine with original features
from scipy.sparse import hstack
X_train_with_title = hstack([X_train, title_train])
X_test_with_title = hstack([X_test, title_test])

print(f"\nOriginal features: {X_train.shape[1]}")
print(f"Title features: {title_train.shape[1]}")
print(f"Total features: {X_train_with_title.shape[1]}")

exp2_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

exp2_results, _ = evaluate_model(exp2_model, X_train_with_title, y_train, X_test_with_title, y_test, name="With Title Features")

print("\nResults:")
for metric, value in exp2_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp2_results)

## 5. Experiment 3: Ensemble Model

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 3 - Ensemble (LR + RF + LightGBM)")
print("=" * 60)

ensemble = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1, class_weight='balanced')),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')),
        ('lgbm', lgb.LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1, class_weight='balanced'))
    ],
    voting='soft',
    n_jobs=-1
)

print("\nTraining ensemble (this may take a few minutes)...")
exp3_results, _ = evaluate_model(ensemble, X_train, y_train, X_test, y_test, name="Ensemble")

print("\nResults:")
for metric, value in exp3_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp3_results)

## 6. Experiment 4: Feature Selection + Title Features (Combined)

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 4 - Feature Selection + Title Features")
print("=" * 60)

# Combine title with original, then select top features
X_train_combined = hstack([X_train, title_train])
X_test_combined = hstack([X_test, title_test])

# Select top 2000 features from combined set
selector_combined = SelectKBest(chi2, k=2000)
X_train_best_combined = selector_combined.fit_transform(X_train_combined, y_train)
X_test_best_combined = selector_combined.transform(X_test_combined)

print(f"\nCombined features: {X_train_combined.shape[1]}")
print(f"Selected best: {X_train_best_combined.shape[1]}")

exp4_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

exp4_results, _ = evaluate_model(exp4_model, X_train_best_combined, y_train, X_test_best_combined, y_test, name="Selection + Title")

print("\nResults:")
for metric, value in exp4_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp4_results)

## 7. Experiment 5: Ensemble + Title Features

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 5 - Ensemble + Title Features")
print("=" * 60)

ensemble_title = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1, class_weight='balanced')),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')),
        ('lgbm', lgb.LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1, class_weight='balanced'))
    ],
    voting='soft',
    n_jobs=-1
)

print("\nTraining ensemble with title features...")
exp5_results, _ = evaluate_model(ensemble_title, X_train_with_title, y_train, X_test_with_title, y_test, name="Ensemble + Title")

print("\nResults:")
for metric, value in exp5_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp5_results)

## 8. Summary Comparison

In [ ]:
print("\n" + "=" * 80)
print("FINAL COMPARISON - All Experiments")
print("=" * 80)

results_df = pd.DataFrame(results_list)
results_df = results_df.round(4)

# Format as percentages
for col in ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC-AUC']:
    results_df[col] = results_df[col].apply(lambda x: f"{x*100:.2f}%")

print("\n", results_df.to_string(index=False))

# Find best F1
print("\n" + "=" * 80)
f1_scores = [r['F1'] for r in results_list]
best_idx = f1_scores.index(max(f1_scores))
best_model = results_list[best_idx]

print(f"\n🏆 BEST MODEL: {best_model['Model']}")
print(f"   F1 Score: {best_model['F1']*100:.2f}%")
print(f"   Improvement over baseline: {(best_model['F1'] - baseline_results['F1'])*100:+.2f} points")
print(f"   ROC-AUC: {best_model['ROC-AUC']*100:.2f}%")
print("=" * 80)

## 9. Recommendation

In [ ]:
print("\n💡 RECOMMENDATION:")
print("-" * 80)

improvement = (best_model['F1'] - baseline_results['F1']) * 100

if improvement > 1.0:
    print(f"✅ Use '{best_model['Model']}' for final model")
    print(f"   Provides {improvement:.2f} point improvement in F1")
    print(f"   New F1: {best_model['F1']*100:.2f}% (was {baseline_results['F1']*100:.2f}%)")
elif improvement > 0:
    print(f"⚠️  '{best_model['Model']}' provides small improvement (+{improvement:.2f} F1)")
    print(f"   Consider if added complexity is worth the gain")
    print(f"   Baseline is already strong at {baseline_results['F1']*100:.2f}% F1")
else:
    print("❌ None of the experiments improved F1")
    print(f"   Stick with baseline: {baseline_results['F1']*100:.2f}% F1")
    print("   Your current model is already well-optimized")

print("\n" + "-" * 80)
print("Next steps:")
print("1. If improvement is significant, retrain with best configuration")
print("2. Update FEATURE_SUMMARY.md with new metrics")
print("3. Save the best model for deployment")
print("4. Document the approach in your final report")

## 10. Save Best Model (Optional)

In [ ]:
# Uncomment to save the best performing model

# import pickle
# 
# if improvement > 1.0:
#     models_dir = Path('../models')
#     models_dir.mkdir(exist_ok=True)
#     
#     # Retrain best model
#     # (Add code here based on which experiment won)
#     
#     # Save model
#     # with open(models_dir / 'best_classifier_optimized.pkl', 'wb') as f:
#     #     pickle.dump(best_trained_model, f)
#     
#     print("✓ Best model saved!")
# else:
#     print("No improvement - keeping baseline model")